In [1]:
import pandas as pd
import psycopg2, re
import pandas.io.sql as psql
from pandasql import sqldf
from sklearn.feature_extraction.text import CountVectorizer


# Enlace con la base de datos de postgresql

# Postgres
# PSQL_HOST = 
# PSQL_PORT = 
# PSQL_USER = 
# PSQL_PASS = 
# PSQL_DB   = 

# Conectarse a la base de datos
# connstr = "host=%s port=%s user=%s password=%s dbname=%s" % (PSQL_HOST, PSQL_PORT, PSQL_USER, PSQL_PASS, PSQL_DB)
# conn = psycopg2.connect(connstr)

In [2]:
#**********************************************Queries

#carros en venta (CARDATA JOIN CARDETAILS ON ID). 

#Extraigo de acá solo los carros que estan disponibles de la página web. 
#* ver parametros de venta: Is_certified, Is_valide, Is_sale, Is_active. 

#en_venta=psql.read_sql("SELECT id FROM kavakcp_website_remote.sell_car_details WHERE Is_certified=1 AND Is_valide=1 AND Is_sale=0 AND Is_active=1",conn)



#detalles caracteristicos de cada versión de carro.

#car_data=psql.read_sql("SELECT id,sku,make,model,year,full_version,style_id as bodytype FROM kavakcp_website_remote.car_data", conn)


# Query: carros en venta JOIN  car data.
# ****No me funcionó aca por cuestiones de la liberia que no realiza joins "extraordinarios".
#en_venta=psql.read_sql("SELECT CARDATA.id,CARDATA.sku,CARDATA.make,CARDATA.model,CARDATA."year",CARDATA.full_version,CARDATA.style_id as bodytype FROM kavakcp_website_remote.car_data as CARDATA JOIN kavakcp_website_remote.sell_car_details as CARDETAILS ON CARDATA.id=CARDETAILS.id WHERE CARDETAILS.Is_certified=1 AND CARDETAILS.Is_valide=1 AND CARDETAILS.Is_sale=0 AND CARDETAILS.Is_active=1", conn)

#-------------------------------------------------------------------------------------------------


In [3]:
muestra=pd.read_csv('car_data.csv',sep=';')

In [4]:
muestra.head()

,ID,Make,Model,Year,Trim,Colors exterior,Colors interior,Body type,Total seating,Length (in),...,Height (in),Wheelbase (in),Ground clearance (in),Drive type,Transmission,Engine type,Fuel type,Domestic / imported,Country of origin,Car classification
0,200729833,BMW,3 Series,2016,320i 4dr Sedan (2.0L 4cyl Turbo 8A),Jet Black,Black Dakota,Sedan,5,182,...,56,111,6,rear wheel drive,8-speed shiftable automatic,gas,premium unleaded (required),Imported,Germany,Compact car
1,200741593,BMW,3 Series,2016,320i xDrive 4dr Sedan AWD (2.0L 4cyl Turbo 8A),Jet Black,Black Dakota,Sedan,5,182,...,57,111,6,all wheel drive,8-speed shiftable automatic,gas,premium unleaded (required),Imported,Germany,Compact car
2,200741592,BMW,3 Series,2016,328i 4dr Sedan (2.0L 4cyl Turbo 6M),Jet Black,Venetian Beige,Sedan,5,182,...,56,111,6,rear wheel drive,6-speed manual,gas,premium unleaded (required),Imported,Germany,Compact car
3,200741591,BMW,3 Series,2016,328i SULEV 4dr Sedan (2.0L 4cyl Turbo 8A),Jet Black,Venetian Beige,Sedan,5,182,...,56,111,6,rear wheel drive,8-speed shiftable automatic,gas,premium unleaded (required),Imported,Germany,Compact car
4,200741590,BMW,3 Series,2016,328i xDrive SULEV 4dr Sedan AWD (2.0L 4cyl Tur...,Jet Black,Venetian Beige,Sedan,5,182,...,57,111,6,all wheel drive,8-speed shiftable automatic,gas,premium unleaded (required),Imported,Germany,Compact car


In [5]:
#**********************************************Variable: Descripción y Decisión.

#Creación de la variable: descripción, la cual, servira como medio de contraste entre lo que se está buscando y lo que 
#existe en la base de datos de Kavak, es decir, el dataframe: muestra.



muestra["descripcion"]=muestra["Make"]+' '+muestra["Model"]+' '+muestra["Year"].astype(str)+' '+muestra["Trim"]+' '+muestra["Colors exterior"]+' '+muestra["Colors interior"]+' '+muestra["Body type"]+' '+muestra["Total seating"].astype(str)+' '+muestra["Length (in)"].astype(str)+' '+muestra["Width (in)"].astype(str)+' '+muestra["Height (in)"].astype(str)+' '+muestra["Wheelbase (in)"].astype(str)+' '+muestra["Ground clearance (in)"].astype(str)+' '+muestra["Drive type"]+muestra["Transmission"].astype(str)+' '+muestra["Engine type"]+' '+muestra["Fuel type"]+' '+muestra["Domestic / imported"]+' '+muestra["Country of origin"]+' '+muestra["Car classification"]

#muestra["descripcion"]="Kavak"+" "+muestra["bodytype"]+" "+ muestra["make"]+" "+muestra["model"]+" "+muestra['year'].astype(str)+" "+ muestra["prueba"] 
muestra.head(1)


#Variable: Decisión, esta creada para almacenar numericamente, cuantas coinciencias hay entre lo que se esta buscando con lo que
#hay en nuestra base de datos.

muestra["decision"]=0



In [6]:
muestra.isnull().values.any()

#muestra.head(1)

False

In [7]:
#-------------------------------------------------------------------------------------------------

#**********************************************Función sistema de recomendación. BÚSQUEDA POR CONTENIDO

def sistema_de_recomendacion(x,y):
    corpus = [x,y]
    vectorizer = CountVectorizer(min_df=2)
    X = vectorizer.fit_transform(corpus)
    
    return ( len( vectorizer.get_feature_names() ) )


muestra.to_csv('muestra_sistema_de_recomendacion.csv',encoding='utf-8-sig')

In [8]:
#x1, significa la publicación donde esta parado el cliente.

x1='BMW 3 Series 2016 320i 4dr Sedan (2.0L 4cyl Turbo 8A) Jet Black Black Dakota Sedan 5 182 71 56 111 6 rear wheel drive8-speed shiftable automatic gas premium unleaded (required) Imported Germany Compact car'

#Llamada al algoritmo: sistema_de_recomendacion.
muestra['decision']=muestra.apply(lambda row: sistema_de_recomendacion(x1, row['descripcion']), axis=1)

#ordenación de los registros desde la mayor coincidencia hasta la menor.
muestra.sort_values(['decision'], ascending=[False]) 

print( "Filtro por contenido" )
print( muestra.sort_values(['decision'], ascending=[False]).head(15) )

# Se recomiendan solo los primeros 15 primeros autos más parecidos en la busqueda por contenido.

Filtro por contenido
           ID Make     Model  Year  \
0   200729833  BMW  3 Series  2016   
3   200741591  BMW  3 Series  2016   
1   200741593  BMW  3 Series  2016   
11  401638449  BMW  3 Series  2016   
5   200741587  BMW  3 Series  2016   
4   200741590  BMW  3 Series  2016   
13  401729246  BMW  3 Series  2018   
10  200741585  BMW  3 Series  2016   
23  401729245  BMW  3 Series  2018   
15  401729243  BMW  3 Series  2018   
6   200741586  BMW  3 Series  2016   
14  401729240  BMW  3 Series  2018   
17  401729238  BMW  3 Series  2018   
2   200741592  BMW  3 Series  2016   
7   200741594  BMW  3 Series  2016   

                                                 Trim  \
0                 320i 4dr Sedan (2.0L 4cyl Turbo 8A)   
3           328i SULEV 4dr Sedan (2.0L 4cyl Turbo 8A)   
1      320i xDrive 4dr Sedan AWD (2.0L 4cyl Turbo 8A)   
11  330e 4dr Sedan (2.0L 4cyl Turbo gas/electric h...   
5                 340i 4dr Sedan (3.0L 6cyl Turbo 8A)   
4   328i xDrive SULEV 4dr Se

In [9]:
#**********************************************Función sistema de recomendación. BÚSQUEDA POR CONOCIMIENTO


#x1,x2,x3,x4 y x5 simulan las navegaciones del cliente.
x1='BMW 3 Series 2018 340i 4dr Sedan (3.0L 6cyl Turbo 8A) Alpine White Black Dakota w/Dark Oyster Highlight Sedan 5 183 71 56 111 6 rear wheel drive8-speed shiftable automatic gas premium unleaded (recommended) Imported Germany Compact car'
x2='Ford F-150 2020 XL 2dr Regular Cab 8 ft. LB (3.3L 6cyl 6A) Iconic Silver Metallic Dark Earth Gray Truck 3 228 80 75 141 9 rear wheel drive6-speed shiftable automatic gas regular unleaded Domestic United States Full-size pickup truck'
x3='Ford F-150 2020 XL 4dr SuperCrew 6.5 ft. SB (2.7L 6cyl Turbo 10A) Iconic Silver Metallic Black Truck 6 244 80 76 157 8 rear wheel drive10-speed shiftable automatic gas regular unleaded Domestic United States Full-size pickup truck'
x4='Toyota Prius 2017 Three 4dr Hatchback (1.8L 4cyl gas/electric hybrid CVT) Blue Crush Metallic Moonstone Hatchback 5 179 69 58 106 5 front wheel drivecontinuously variable-speed automatic hybrid regular unleaded Imported Japan Compact car'
x5='BMW 3 Series 2016 328d xDrive 4dr Sedan AWD (2.0L 4cyl Turbodiesel 8A) Jet Black Venetian Beige Sedan 5 182 71 57 111 6 all wheel drive8-speed shiftable automatic diesel diesel fuel Imported Germany Compact car'


In [10]:
#Se transforman esas navegaciones a un dataframe
navegacion=pd.DataFrame(
    {"seleccion":[x1,x2,x3,x4,x5], 
     "stock_id":["3795","3848","3703","3723","3823"]},
			columns=['seleccion', 'stock_id']
			)

In [11]:
#Se extrae la columna seleción y se convierte en un vector.
text=navegacion["seleccion"]

In [12]:
#Aplicación de la función de vectorización (Filtro por contenido)
corpus = text
vectorizer = CountVectorizer(min_df=2)
X = vectorizer.fit_transform(corpus)
resultado=vectorizer.get_feature_names()

#Se reacomodan las palabras más comunes.
busqueda_por_conocimiento = ' '.join(map(str, resultado))
#print(busqueda_por_conocimiento)

In [13]:
busqueda_por_conocimiento

'0l 111 150 2020 4cyl 4dr 6cyl 71 80 8a automatic black bmw car compact dark domestic drive8 ford ft full gas germany iconic imported metallic pickup rear regular sedan series shiftable silver size speed states truck turbo united unleaded wheel xl'

In [14]:
#Se inicializa la variable muestra.
muestra["decision"]=0

#x10, representa lo que indica la variable: busqueda_por_conocimiento
x10=busqueda_por_conocimiento
muestra['decision']=muestra.apply(lambda row: sistema_de_recomendacion(x10, row['descripcion']), axis=1)
muestra.sort_values(['decision'], ascending=[False]) 

,ID,Make,Model,Year,Trim,Colors exterior,Colors interior,Body type,Total seating,Length (in),...,Ground clearance (in),Drive type,Transmission,Engine type,Fuel type,Domestic / imported,Country of origin,Car classification,descripcion,decision
39,401822562,Ford,F-150,2020,XL 4dr SuperCab 8 ft. LB (2.7L 6cyl Turbo 10A),Iconic Silver Metallic,Dark Earth Gray,Truck,6,251,...,9,rear wheel drive,10-speed shiftable automatic,gas,regular unleaded,Domestic,United States,Full-size pickup truck,Ford F-150 2020 XL 4dr SuperCab 8 ft. LB (2.7L...,28
52,401822594,Ford,F-150,2020,XL 4dr SuperCrew 6.5 ft. SB (2.7L 6cyl Turbo 10A),Iconic Silver Metallic,Black,Truck,6,244,...,8,rear wheel drive,10-speed shiftable automatic,gas,regular unleaded,Domestic,United States,Full-size pickup truck,Ford F-150 2020 XL 4dr SuperCrew 6.5 ft. SB (2...,28
55,401822604,Ford,F-150,2020,XLT 4dr SuperCrew 5.5 ft. SB (3.3L 6cyl 6A),Iconic Silver Metallic,Black,Truck,6,232,...,9,rear wheel drive,6-speed shiftable automatic,gas,regular unleaded,Domestic,United States,Full-size pickup truck,Ford F-150 2020 XLT 4dr SuperCrew 5.5 ft. SB (...,26
42,401822572,Ford,F-150,2020,XLT 4dr SuperCab 6.5 ft. SB (3.3L 6cyl 6A),Iconic Silver Metallic,Black,Truck,6,232,...,9,rear wheel drive,6-speed shiftable automatic,gas,regular unleaded,Domestic,United States,Full-size pickup truck,Ford F-150 2020 XLT 4dr SuperCab 6.5 ft. SB (3...,26
40,401822570,Ford,F-150,2020,XL 4dr SuperCab 4WD 6.5 ft. SB (3.3L 6cyl 6A),Iconic Silver Metallic,Black,Truck,6,232,...,9,four wheel drive,6-speed shiftable automatic,gas,regular unleaded,Domestic,United States,Full-size pickup truck,Ford F-150 2020 XL 4dr SuperCab 4WD 6.5 ft. SB...,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,401611902,Toyota,Prius,2016,Two Eco 4dr Hatchback (1.8L 4cyl gas/electric ...,Hypersonic Red,Black,Hatchback,5,179,...,5,front wheel drive,continuously variable-speed automatic,hybrid,regular unleaded,Imported,Japan,Compact car,Toyota Prius 2016 Two Eco 4dr Hatchback (1.8L ...,12
80,401709372,Toyota,Prius,2017,One 4dr Hatchback (1.8L 4cyl gas/electric hybr...,Hypersonic Red,Moonstone,Hatchback,5,179,...,5,front wheel drive,continuously variable-speed automatic,hybrid,regular unleaded,Imported,Japan,Compact car,Toyota Prius 2017 One 4dr Hatchback (1.8L 4cyl...,11
77,401611904,Toyota,Prius,2016,Three Touring 4dr Hatchback (1.8L 4cyl gas/ele...,Hypersonic Red,Harvest Beige,Hatchback,5,179,...,5,front wheel drive,continuously variable-speed automatic,hybrid,regular unleaded,Imported,Japan,Compact car,Toyota Prius 2016 Three Touring 4dr Hatchback ...,11
79,401611906,Toyota,Prius,2016,Four Touring 4dr Hatchback (1.8L 4cyl gas/elec...,Hypersonic Red,Harvest Beige,Hatchback,5,179,...,5,front wheel drive,continuously variable-speed automatic,hybrid,regular unleaded,Imported,Japan,Compact car,Toyota Prius 2016 Four Touring 4dr Hatchback (...,11
